In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import gc,os,sys
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler

import warnings
warnings.simplefilter('ignore')

from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from mlxtend.classifier import StackingCVClassifier

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_colwidth', 30)

In [ ]:
RANDOM_SEED = 2021
PROBAS = True
FOLDS = 5
N_ESTIMATORS = 1000

TARGET = 'Survived'

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

pseudo_labels = pd.read_csv('../input/pseudo-label/pseudo_label.csv')
test_df[TARGET] = pseudo_labels[TARGET]

temp = pd.concat([train_df, test_df]).reset_index(drop=True)

In [ ]:
temp['Age'] = temp['Age'].fillna(temp['Age'].median())

temp["Age_bucket"] = pd.cut(temp['Age'], 9,
                            labels = ["0-9", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80-89"])

temp['Embarked'] = temp['Embarked'].fillna('N')

temp['CabinNotAlloted'] = temp.Cabin.isna().astype(int)

temp['Cabin'] = temp.Cabin.str[:1]
temp['Cabin'].fillna('X')

temp['TicketType'] = temp.Ticket.fillna('XX')
temp['TicketType'] = temp.TicketType.map(lambda x: str(x).split()[0] if len(str(x).split()) > 1 else 'XX')
temp['TicketType'] = temp['TicketType'].str.lower()

import re
temp['TicketType'] = temp.TicketType.map(lambda x: re.sub("[^\w\s]+","",x))

temp['TT_bucket'] = temp.TicketType.map(lambda x:0 if x == 'pc' else 3 if x in ['stono', 'stono2', 'sotono2', 'stonoq', 'aq3', 'a', 'a5', 'sotonoq', 'fa', 'ca'] \
                                          else 2 if x in ['fcc', 'scow', 'caston', 'wc', 'c', 'wep', 'swpp', 'ppp', 'scah', 'soc', 'sopp'] else 1)

FareByClass = pd.crosstab(index = temp.Pclass, columns = 'MedianFare', values = np.log(1+temp.Fare), aggfunc = 'median').to_dict()['MedianFare']
FareByClass

temp['LnFare'] = np.log(1+temp['Fare'])

temp['LnFare'].fillna(temp.Pclass.map(FareByClass), inplace = True)

temp['FamilySize'] = temp['SibSp'] + temp['Parch'] + 1

def family_size(x):
    if x == 1:
        return "alone"
    else:
        return "notalone"
    
temp['Group'] = temp['FamilySize'].apply(family_size)
print(temp['Group'].value_counts())    


In [ ]:
temp.columns

In [ ]:
label_cols = ['Sex','CabinNotAlloted', 'TT_bucket', 'Cabin','Group']
onehot_cols = ['Age_bucket','Embarked']
numerical_cols = ['LnFare','Age','Pclass']

In [ ]:
def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c.astype(str))

In [ ]:
onehot_encoded_df = pd.get_dummies(temp[onehot_cols])
label_encoded_df = temp[label_cols].apply(label_encoder)
numerical_df = pd.DataFrame(temp[numerical_cols], columns=numerical_cols)
target_df = temp['Survived']

all_df = pd.concat([numerical_df, label_encoded_df, onehot_encoded_df, target_df], axis=1)

In [ ]:
lgbm_parameters = {
  'metric' : 'binary_logloss',
  'n_estimators' : N_ESTIMATORS,
  'objective' : 'binary',
  'random_state' : RANDOM_SEED,
  'learning_rate' : 0.01,  
  'colsample_bytree': 0.6283725788215941,
  'max_bin': int(round(15.826197551963968)),
  'max_depth': int(round(39.32209311790955)),
  'min_child_weight': 44.95339851660889,
  'min_split_gain': 0.04358718365142237,
  'num_leaves': int(round(24.715504910160405)),
  'reg_alpha': 0.4127198530404361,
  'reg_lambda': 0.0006949333245371281,
  'subsample': 0.7192205961769677,
  'subsample_freq': int(round(13.984681107001574))}

cat_parameters =  {
  'bootstrap_type' : 'Poisson',
  'loss_function' : 'Logloss',
  'eval_metric' : 'Logloss',
  'random_seed' : RANDOM_SEED,
  'task_type': 'GPU',
  'learning_rate' : 0.01,
  'n_estimators' : N_ESTIMATORS, 
  'bagging_temperature': 534.445170361156,
  'border_count': int(round(230.32755580650806)),
  'depth': int(round(5.969930611242375)),
  'learning_rate': 0.01966964700090523,
  'min_data_in_leaf': 2.208728103621775}




In [ ]:
cl1 = CatBoostClassifier(**cat_parameters, verbose = None, logging_level = 'Silent')
cl2 = LGBMClassifier(**lgbm_parameters)


In [ ]:
mlr = LogisticRegression()

In [ ]:
scl = StackingCVClassifier(classifiers= [cl1, cl2], #[cl1, cl2, cl3, cl4, cl5, cl6, cl7, cl8]
                            meta_classifier = mlr, # use meta-classifier
                            use_probas = PROBAS,   # use_probas = True/False
                            random_state = RANDOM_SEED)

NUM_CLAS = 4 # classifiers (l1) + stacked (meta-classifier)

classifiers = {"CatBoost": cl1,
               "LGBM": cl2,
               "Stacked": scl}

In [ ]:
X = all_df.drop([TARGET], axis = 1)
y = all_df[TARGET]

print (f'X:{X.shape} y: {y.shape} \n')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = RANDOM_SEED)
print (f'X_train:{X_train.shape} y_train: {y_train.shape}')
print (f'X_test:{X_test.shape} y_test: {y_test.shape}')

In [ ]:
test = all_df[len(train_df):].drop([TARGET], axis = 1)
print (f'test:{test.shape}')

Train classifiers

In [ ]:
print(">>>> Training started <<<<")
for key in classifiers:
    classifier = classifiers[key]
    scores = model_selection.cross_val_score(classifier, X_train, y_train, cv=FOLDS, scoring='accuracy')
    print("[%s] = accuracy: %0.2f" % (key, scores.mean()))
    classifier.fit(X_train, y_train)
    
    classifiers[key] = classifier

Predict and validate (AUC)

In [ ]:
preds = pd.DataFrame()

for key in classifiers:
    y_pred = classifiers[key].predict_proba(X_test)[:,1]
    preds[f"{key}"] = y_pred
    auc = metrics.roc_auc_score(y_test, y_pred)
    print(f"{key} --> AUC: {auc:.3f}")
    
preds[TARGET] = pd.DataFrame(y_test).reset_index(drop=True)

In [ ]:
test_preds = classifiers['Stacked'].predict_proba(test)[:,1]

In [ ]:
threshold = pd.Series(test_preds).sort_values(ascending = False).head(34911).values[-1]
print(f"Current threshold is: {threshold}")

In [ ]:
submission['submit_1'] = (test_preds > threshold).astype(int)
submission['submit_1'].mean()

In [ ]:
submission['submit_2'] = pd.read_csv("../input/for-score/dae.csv")[TARGET]
submission['submit_3'] = pseudo_labels[TARGET]

In [ ]:
submission[[col for col in submission.columns if col.startswith('submit_')]].sum(axis = 1).value_counts()


In [ ]:
submission[TARGET] = (submission[[col for col in submission.columns if col.startswith('submit_')]].sum(axis=1) >= 2).astype(int)
submission[TARGET].mean()

In [ ]:
submission[['PassengerId', TARGET]].to_csv("submission.csv", index = False)